<a href="https://colab.research.google.com/github/Ritika-Gada/verbose-memory/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:
1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that maerks whether the news article is real or fake;

1: Fake news

0: real news


Importing the Dependencies

In [3]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
#printing the stop words in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing 

In [6]:

#loading the dataset to a pandas DataFrame
newsdata = pd.read_csv('/content/train.csv')

In [7]:
 
  newsdata.shape

(20800, 5)

In [8]:
newsdata.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
 newsdata.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
#replacing the null values with empty string
newsdata = newsdata.fillna('')

In [11]:
#merging the author names and news title
newsdata['content'] = newsdata['author']+' '+ newsdata['title']

In [12]:
print(newsdata['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [13]:
#seperating the data and label
X = newsdata.drop('label',axis=1)
Y = newsdata['label']


In [14]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming : 
Stemming is the proces of reducing a word to its root word. 
Example : actor, actress, acting --> act

In [15]:
 port_stem = PorterStemmer()

In [24]:
def stemming(content) :
  stemmed_content = re.sub('[^a-zA-z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content  

In [25]:
newsdata['content'] = newsdata['content'].apply(stemming)

In [26]:
print(newsdata['content'])

0        darrelllucushousedemaidewedidntevenseecomeysle...
1        danieljflynnflynnhillaryclintonbigwomanoncampu...
2               consortiumnewscomwhythetruthmightgetyoufir
3        jessicapurkisscivilianskilledinsingleusairstri...
4        howardportnoyiranianwomanjailedforfictionalunp...
                               ...                        
20795    jeromehudsonrappertitrumpaposterchildforwhites...
20796    benjaminhoffmannflplayoffsschedulematchupsando...
20797    michaeljdelamercedandrachelabramsmacysissaidto...
20798    alexansarynatorussiatoholdparallelexercisesinb...
20799                            davidswansonwhatkeepsthef
Name: content, Length: 20800, dtype: object


In [19]:
#seperating the data and label
X = newsdata['content'].values
Y = newsdata['label'].values

In [20]:
print(X)
print(Y)

['darrelllucushousedemaidewedidntevenseecomeysletteruntiljasonchaffetztweetedit'
 'danieljflynnflynnhillaryclintonbigwomanoncampusbreitbart'
 'consortiumnewscomwhythetruthmightgetyoufir' ...
 'michaeljdelamercedandrachelabramsmacysissaidtoreceivetakeoverapproachbyhudsonsbaythenewyorktim'
 'alexansarynatorussiatoholdparallelexercisesinbalkan'
 'davidswansonwhatkeepsthefal']
[1 0 1 ... 0 1 1]


In [27]:
Y.shape

(20800,)

In [30]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [31]:
print(X)

  (0, 5210)	1.0
  (1, 4898)	1.0
  (2, 4551)	1.0
  (3, 9761)	1.0
  (4, 7983)	1.0
  (5, 4996)	1.0
  (6, 12013)	1.0
  (7, 1010)	1.0
  (8, 6809)	1.0
  (9, 12909)	1.0
  (10, 85)	1.0
  (11, 4203)	1.0
  (12, 15651)	0.6926972991877945
  (12, 1165)	0.72122843239707
  (13, 9115)	1.0
  (14, 1697)	1.0
  (15, 8745)	1.0
  (16, 13043)	1.0
  (17, 17480)	1.0
  (18, 18140)	1.0
  (19, 9289)	1.0
  (20, 14194)	1.0
  (21, 9671)	1.0
  (22, 15118)	1.0
  (23, 12718)	1.0
  :	:
  (20775, 1823)	1.0
  (20776, 8939)	1.0
  (20777, 409)	1.0
  (20778, 14005)	1.0
  (20779, 18541)	1.0
  (20780, 4167)	1.0
  (20781, 7757)	1.0
  (20782, 18172)	1.0
  (20783, 3728)	1.0
  (20784, 5554)	1.0
  (20785, 1572)	1.0
  (20786, 7472)	1.0
  (20787, 8253)	1.0
  (20788, 9871)	1.0
  (20789, 19697)	1.0
  (20790, 288)	1.0
  (20791, 4865)	1.0
  (20792, 10248)	1.0
  (20793, 16444)	1.0
  (20794, 11876)	1.0
  (20795, 9686)	1.0
  (20796, 2366)	1.0
  (20797, 13154)	1.0
  (20798, 890)	1.0
  (20799, 5476)	1.0


Splitting the dataset to training and test data

In [32]:
xtrain,xtest,ytrain,ytest = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state =2)

Training the Model : Logistic Regression


In [33]:
model = LogisticRegression()

In [34]:
model.fit(xtrain,ytrain)

LogisticRegression()

Evalution

Accuracy Score

In [35]:
#accuracy score on training data
xtrain_pred = model.predict(xtrain)
training_data_acc = accuracy_score(xtrain_pred, ytrain)

In [36]:
print('Accuracy Score of the training data : ', training_data_acc)

Accuracy Score of the training data :  0.9927283653846154


In [38]:
#accuracy score of the test data
xtest_pred = model.predict(xtest)
test_data_acc = accuracy_score(xtest_pred, ytest)
print('Accuracy score of the test data : ', test_data_acc)

Accuracy score of the test data :  0.5377403846153846


In [39]:
xnew = xtest[3]
prediction = model.predict(xnew)
print(prediction)

if (prediction[0] ==0):
  print('The news is Real')
else:
  print('The news is fake')

[0]
The news is Real


In [40]:
print(ytest[3])

0
